In [2]:
import numpy as np

def recommend(
    query,
    df,
    cosine_sim,      # title/text similarity
    genre_sim,       # genre one-hot similarity
    rating_sim,      # rating similarity
    year_sim,        # year similarity
    n=10,
    w_title=0.45,
    w_genre=0.25,
    w_rating=0.15,
    w_year=0.10,
    w_cluster=0.05   # boost for same-cluster books
):
    """
    Unified hybrid recommendation system:
    - Fuzzy title matching
    - Title (TF-IDF) similarity
    - Genre similarity
    - Rating similarity
    - Published year similarity
    - Cluster-aware boosting
    """

    # ---------------------------------------------------------
    # 1. Fuzzy match the title
    # ---------------------------------------------------------
    q = query.lower().strip()
    matches = df[df["title"].str.lower().str.contains(q)]

    if matches.empty:
        return f"No books found for '{query}'. Try another keyword."

    # take first match
    idx = matches.index[0]
    book_cluster = df.loc[idx, "cluster"]

    # ---------------------------------------------------------
    # 2. Extract all similarity vectors for the book
    # ---------------------------------------------------------
    sim_title_vec  = cosine_sim[idx]
    sim_genre_vec  = genre_sim[idx]
    sim_rating_vec = rating_sim[idx]
    sim_year_vec   = year_sim[idx]

    # ---------------------------------------------------------
    # 3. Normalize weights (optional but improves stability)
    # ---------------------------------------------------------
    w_sum = w_title + w_genre + w_rating + w_year + w_cluster
    w_title  /= w_sum
    w_genre  /= w_sum
    w_rating /= w_sum
    w_year   /= w_sum
    w_cluster /= w_sum

    # ---------------------------------------------------------
    # 4. Compute hybrid score for all books
    # ---------------------------------------------------------
    hybrid_score = (
        w_title  * sim_title_vec +
        w_genre  * sim_genre_vec +
        w_rating * sim_rating_vec +
        w_year   * sim_year_vec
    )

    # ---------------------------------------------------------
    # 5. Apply cluster-aware boosting
    # ---------------------------------------------------------
    same_cluster = (df["cluster"] == book_cluster).astype(int)
    hybrid_score += w_cluster * same_cluster

    # ---------------------------------------------------------
    # 6. Sort by final hybrid score
    # ---------------------------------------------------------
    indices_sorted = np.argsort(hybrid_score)[::-1]

    # remove the input book itself
    indices_sorted = [i for i in indices_sorted if i != idx]

    # take top N
    top_indices = indices_sorted[:n]

    # ---------------------------------------------------------
    # 7. Return final results
    # ---------------------------------------------------------
    return df.iloc[top_indices][[
        "title",
        "author",
        "avg_rating",
        "rating_count",
        "year_published",
        "all_genres",
        "cluster"
    ]]


In [3]:
import streamlit as st
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# -------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------
@st.cache_data
def load_data():
    df = pd.read_csv("../data/clean/books_cluster.csv")   # <-- change to your file path
    return df

df = load_data()

# -------------------------------------------------------------
# 2. LOAD PRECOMPUTED MATRICES
# -------------------------------------------------------------
@st.cache_resource
def compute_similarity_matrices(df):

    # ---- title similarity (TF-IDF) ----
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf.fit_transform(df["features_text"])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # ---- genre similarity ----
    genre_cols = [
        "activism","adult","bestseller","children/young adult","fantasy","fiction",
        "guide","history","inspirational","literature","non-fiction","other",
        "poetry","romance","science","spiritual/religious","sport"
    ]
    genre_matrix = df[genre_cols].values
    genre_sim = cosine_similarity(genre_matrix)

    # ---- rating similarity ----
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    df["rating_scaled"] = scaler.fit_transform(df[["avg_rating"]])
    rating_sim = cosine_similarity(df[["rating_scaled"]])

    # ---- year similarity ----
    df["year_published"] = pd.to_numeric(df["year_published"], errors="coerce")
    df["year_published"] = df["year_published"].fillna(df["year_published"].median()).astype(int)
    df["year_scaled"] = scaler.fit_transform(df[["year_published"]])
    year_sim = cosine_similarity(df[["year_scaled"]])

    return cosine_sim, genre_sim, rating_sim, year_sim

cosine_sim, genre_sim, rating_sim, year_sim = compute_similarity_matrices(df)

# -------------------------------------------------------------
# 3. UNIFIED HYBRID RECOMMENDER FUNCTION
# -------------------------------------------------------------
def recommend(
    query,
    df,
    cosine_sim, genre_sim, rating_sim, year_sim,
    n=10,
    w_title=0.45, w_genre=0.25, w_rating=0.15, w_year=0.10, w_cluster=0.05
):

    # Fuzzy search
    q = query.lower().strip()
    matches = df[df["title"].str.lower().str.contains(q)]
    if matches.empty:
        return None, f"No books found matching '{query}'."

    idx = matches.index[0]
    book_cluster = df.loc[idx, "cluster"]

    # Similarity vectors
    sim_title = cosine_sim[idx]
    sim_genre = genre_sim[idx]
    sim_rating = rating_sim[idx]
    sim_year = year_sim[idx]

    # Normalize weights
    w_sum = w_title + w_genre + w_rating + w_year + w_cluster
    w_title /= w_sum
    w_genre /= w_sum
    w_rating /= w_sum
    w_year /= w_sum
    w_cluster /= w_sum

    # Hybrid score
    hybrid_score = (
        w_title  * sim_title +
        w_genre  * sim_genre +
        w_rating * sim_rating +
        w_year   * sim_year
    )

    # Cluster boost
    same_cluster = (df["cluster"] == book_cluster).astype(int)
    hybrid_score += w_cluster * same_cluster

    # Sort recommendations
    indices_sorted = np.argsort(hybrid_score)[::-1]
    indices_sorted = [i for i in indices_sorted if i != idx]
    top_indices = indices_sorted[:n]

    results = df.iloc[top_indices][[
        "title", "author", "avg_rating",
        "rating_count", "year_published", "all_genres", "cluster"
    ]]

    return results, None

# -------------------------------------------------------------
# 4. STREAMLIT UI
# -------------------------------------------------------------
st.title("📚 Hybrid Book Recommendation System")
st.write("A multi-signal recommender using title, genre, rating, year, and clusters.")

tab1, tab2 = st.tabs(["🔍 Recommendations", "📊 Dataset Information"])

# ---------------------- TAB 1 ------------------------------
with tab1:

    user_query = st.text_input("Search for a book title or keyword:")

    st.subheader("Weights")
    w_title  = st.slider("Title Similarity",   0.0, 1.0, 0.45)
    w_genre  = st.slider("Genre Similarity",   0.0, 1.0, 0.25)
    w_rating = st.slider("Rating Influence",   0.0, 1.0, 0.15)
    w_year   = st.slider("Year Influence",     0.0, 1.0, 0.10)
    w_cluster = st.slider("Cluster Boost",     0.0, 1.0, 0.05)

    n = st.slider("Number of Recommendations:", 3, 20, 10)

    if user_query:
        results, error = recommend(
            user_query, df,
            cosine_sim, genre_sim, rating_sim, year_sim,
            n=n,
            w_title=w_title, w_genre=w_genre,
            w_rating=w_rating, w_year=w_year,
            w_cluster=w_cluster
        )

        if error:
            st.error(error)
        else:
            st.subheader("Recommended Books")
            st.dataframe(results)

# ---------------------- TAB 2 ------------------------------
with tab2:

    st.subheader("Dataset Overview")
    st.write(df.describe())

    st.subheader("Genre Distribution")
    genre_counts = df[
        genre_cols := [
            "activism","adult","bestseller","children/young adult","fantasy","fiction",
            "guide","history","inspirational","literature","non-fiction","other",
            "poetry","romance","science","spiritual/religious","sport"
        ]
    ].sum().sort_values(ascending=False)
    st.bar_chart(genre_counts)

    st.subheader("Cluster Sizes")
    st.bar_chart(df["cluster"].value_counts().sort_index())


2025-11-14 03:49:27.191 No runtime found, using MemoryCacheStorageManager
2025-11-14 03:49:27.192 No runtime found, using MemoryCacheStorageManager
2025-11-14 03:49:27.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.222 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 03:49:27.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor